# Low-Level Driver Demo
This is the low-level driver to make a bridge between the µc and Python.  
Therefore, the interface is not convenient and is not intended to be used as is but rather to implement a higher-level driver.  

----

In [1]:
import lldriver as lld

help(lld.LLDriver)

Help on class LLDriver in module lldriver:

class LLDriver(builtins.object)
 |  LLDriver(pid=22336)
 |  
 |  Low-Level driver for the Awesome Array Python Driver.
 |  
 |  
 |  ...
 |  Attributes
 |  -----------
 |  ser : serial.Serial
 |          serial port associated with the µc
 |  
 |  Methods defined here:
 |  
 |  __del__(self)
 |      Closes the serial port if still open.
 |  
 |  __init__(self, pid=22336)
 |      Creates the driver.
 |      
 |      Details:
 |              It will search for the µc using the PID value 'DEFAULT_PID' or the one provided in argument.
 |              Takes the first found if many have the same PID.
 |              RAISE if not found.
 |      
 |      Arguments:
 |              pid: optional, the pid to search for.
 |  
 |  commands(name=None)
 |      [STATIC] Returns a list of the available µc commands or its hex code if a name is provided.
 |  
 |  list_ports()
 |      [STATIC] Returns a list of all the serial ports recognized by the OS.
 |  
 |

In [2]:
print("\n---- List of available commands ----")
for com in lld.LLDriver.commands():
    print("*", com)


---- List of available commands ----
* SET_SR
* CLK
* GET_CTL
* DBG:PING
* DBG:LED


## Description of the commands
* `SET_SR`:
    Set Shift Register input bit
    * Argument 0: ID of the Shift Register (see Shift Register IDs below)  
    * Argument 1: Input bit state (0 = RESET, otherwise = SET)  
    
    
* `CLK`:
    Clock once the Shift Registers
    
    
* `GET_CTL`:
    Get Shift Registers sanity bit
    * Argument 0: ID of the Shift Register (see Shift Register IDs below)  
    =======
    * Returns the bit state (0 = RESET, 1 = SET)

    
* `DBG:PING`:
    Sends back the provided argument
    * Argument 0: byte to echo
    =======
    * Returns the byte provided
    
    
* `DBG:LED`:
    Turn on the three board LEDs for 2s
    * Returns b'Hello, C2N!'

### Shift Register IDs
| SR    | ID  |
| ----- | --- |
| `WLE` | 0   |
| `WLO` | 1   |
| `SL`  | 2   |
| `BL`  | 3   |
| `BLb` | 4   |

----


In [3]:
print("---- List of available ports ----")
lld.LLDriver.print_ports()

---- List of available ports ----
COM3 - Intel(R) Active Management Technology - SOL (COM3) | PID:  None
COM1 - Communications Port (COM1) | PID:  None
COM5 - STMicroelectronics STLink Virtual COM Port (COM5) | PID:  14155
COM6 - USB Serial Device (COM6) | PID:  22336



We see that the µc is at the port COM6.

In [4]:
driver = lld.LLDriver()
print("Port selected by the LLDriver:", driver.ser.port)

Port selected by the LLDriver: COM6


The LLDriver selected it by default.  
Now for the commands.

In [5]:
driver.send_command('DBG:LED')
driver.read()

b'Hello, C2N!'

In [6]:
driver.send_command('DBG:PING', b'!')
driver.read()

b'!'

In [7]:
# Check WLE sanity | TO EXEC

driver.send_command('SET_SR', b'\x00', b'\x01')
driver.send_command('CLK')
driver.send_command('SET_SR', b'\x00', b'\x00')

for _ in range(63):
	driver.send_command('CLK')

driver.send_command('GET_CTL', b'\x00')
print("This should give b'\x01':", driver.read())